In [1]:
from collections import OrderedDict
import time

from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus

from random_lp.lp_random_gen import create_models

In [2]:
DIR = 'TEST_DATA' + "/" + time.strftime("%d_%m_%Y") # 27.03.2021
Q_SEED = 10598 # as used in most issues
aqua_globals.random_seed = Q_SEED
shots = 512

In [3]:
qps = create_models(DIR + "/DENSE/", penalty=1)
print(qps.keys())
qps_sparse = create_models(DIR + "/SPARSE/", penalty=1)
qps.update(qps_sparse)
print(qps_sparse.keys())

odict_keys(['test_3', 'test_5', 'test_7', 'test_9', 'test_11', 'test_13', 'test_15'])
odict_keys(['test_2', 'test_4', 'test_6'])


In [4]:

optimizer = COBYLA() 
cplex = CplexOptimizer()

backend = BasicAer.get_backend('qasm_simulator')
max_qubits = backend.configuration().to_dict()['n_qubits']

quantum_instance = QuantumInstance(backend,
                                   seed_simulator=Q_SEED,
                                   seed_transpiler=Q_SEED,
                                   shots=shots) 
qaoa_mes = QAOA(quantum_instance=quantum_instance, optimizer=optimizer)

In [5]:
results = OrderedDict()
for qp_name, qp in qps.items() :
    print(qp_name)
    print("number of qubits: ", qp.qubo.get_num_vars())
    if qp_name in qps_sparse.keys():
        print("Sparse")
    if qp.qubo.get_num_vars() > max_qubits:
        print("Too many qubits!")
        continue
        
    qaoa = MinimumEigenOptimizer(qaoa_mes)
    res = qaoa.solve(qp)
    results[qp_name] = res
    if res.status == OptimizationResultStatus.SUCCESS:
        print("success")
        cres  = cplex.solve(qp)
        if res.fval != cres.fval:
            print("\n optimal value QAOA   "+str(res.fval) + "  ,  cplex:"+ str(cres.fval))
    else:
        print(res.status)

test_3
number of qubits:  5
/home/tom/Bachelorarbeit/GitHub/qiskit-terra/qiskit/quantum_info/operators/symplectic/pauli.py:185: DeprecationWarning: Initializing Pauli from `Pauli(z=z, x=x)` kwargs is deprecated as of version 0.17.0 and will be removed no earlier than 3 months after the release date. Use tuple initialization `Pauli((z, x))` instead.
  base_z, base_x, base_phase = self._from_array_deprecated(z, x)
/home/tom/Bachelorarbeit/GitHub/qiskit-aqua/qiskit/aqua/operators/primitive_ops/pauli_op.py:93: DeprecationWarning: `kron` is deprecated and will be removed no earlier than 3 months after the release date of Qiskit Terra 0.17.0. Use `expand` instead, but note this does not change the operator in-place.
  return PauliOp(op_copy.kron(self.primitive), coeff=self.coeff * other.coeff)
success
test_5
number of qubits:  10
success

 optimal value QAOA   -1.0  ,  cplex:-2.0
test_7
number of qubits:  14
success

 optimal value QAOA   -3.0  ,  cplex:-8.0
test_9
number of qubits:  15
test

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.